In [5]:
from __future__ import print_function
from builtins import super
"""
Implementation of Block Neural Autoregressive Flow
http://arxiv.org/abs/1904.04676
"""
import torch.nn.functional as F
import random
import torch
torch.set_printoptions(profile="full")
import copy
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as D
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import warnings
import torchvision
print ("torch",torch.__version__)
print ("torchvision",torchvision.__version__)
warnings.filterwarnings("ignore")

import sys
import array
import math
import os
import time
import argparse
import pprint
import scipy
from scipy import special
from functools import partial
import json
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from tqdm import tqdm
np.set_printoptions(threshold=sys.maxsize)
from sklearn.manifold import TSNE
import csv
from torch.utils import data

from tqdm import trange





torch 1.8.1
torchvision 0.2.2


In [6]:

# --------------------
# Data
# --------------------
matplotlib.use('tkagg')
class sample:
    def __init__(self, nchan ,biasmag, batch_size, topred, seed=None):
        self.biasmag=biasmag
        self.batch_size=batch_size
        self.nchan=nchan
        self.seed=seed
        self.topred=topred
    def sreturn(self):
        xvals=[]
        yvals=[]
        ychan=[]
        if (self.seed==None):
                rng = np.random
        else:
                rng = np.random.RandomState(self.seed)
        #print (-1,1,(self.batch_size,self.nchan))
        randonarr=rng.uniform(-1,1,(self.batch_size,self.nchan))*self.biasmag
        #randonarr[:,1]=1.0*randonarr[:,0]+0.0*randonarr[:,1]


        ntimep = 100+self.topred
        for bb in range(self.batch_size):


                curx=[]
                cury=[]
                prevarrand=np.array([])
                perm=np.arange(4)
                np.random.shuffle(perm)
                #print(perm)
                ychan.append(perm)
                for nn in range(self.nchan):
                        sinfac=rng.uniform(0.2,1.)*1.2
                        sinarr=np.sin(np.arange(ntimep)*sinfac)*5
                        cosfac=rng.uniform(0.2,1.)*1.2
                        cosarr=np.cos(np.arange(ntimep)*cosfac)*5
                        #print(sinfac)
                        #print(cosfac)
                        corrnoise=False
                        arrand = rng.uniform(-1,1,ntimep)
                        if corrnoise:
                                if prevarrand.size==0:
                                        arrand = rng.uniform(-1,1,ntimep)
                                else:
                                        arrand = rng.uniform(-1,1,ntimep)
                                        arrand[nn*5:]=prevarrand[0:-1*nn*5]
                                cs=np.cumsum(arrand)
                        else:
                                arrand = rng.uniform(-1,1,ntimep)
                                cs=np.cumsum(arrand+randonarr[bb][nn])+sinarr+cosarr

                        #print (sinarr) 
                        curx.append(cs[:-self.topred])
                        cury.append(cs[-self.topred:])
                        #curx.append(np.cumsum(arrand+randonarr[bb][0]))
                        prevarrand=arrand
                curx=np.array(curx)
                cury=np.array(cury)
                #print (ychan[-1])
                #print (curx[:,0])

                curx=curx[ychan[-1]]
                #curx=np.transpose(curx,ychan[-1])
                #print (curx[:,0])
                #print ()
                cury=cury[ychan[-1]]
     
                xvals.append((curx))
                yvals.append((cury))
                #print (np.array(xvals).shape)
                #print (np.array(yvals).shape)
                #plt.plot(curx)
                #plt.xlabel("steps")
                #plt.ylabel("position")
                #plt.title("randdwalk")
                #plt.show()
                #sys.exit()
                #input("Press Enter to continue...")
        return np.array(xvals, dtype="float32"),np.array(yvals, dtype="float32"),np.array(ychan, dtype="int64")







In [7]:
class stockdata:
    def __init__(self,totsym=-1):
        
        self.tottime=None
        self.values=None
        self.range=None
        self.corrs=None
        self.symnames=[]
        dirname="filtered/"
        alldown=os.listdir(dirname)
        self.corrs={}
        self.sigmadist=[]

        maxsym=len(alldown)
        print("Loading...")
        for ialld,alld in enumerate (alldown):
            
            if totsym>0:
                maxsym=totsym
                if ialld>totsym:
                    break
            if ialld%500==0:
                print(ialld,"/",maxsym)
            if alld.find("corr_")!=-1:
                continue

            self.symnames.append(alld.replace(".json",""))
            curdf=pd.DataFrame(pd.read_json(dirname+alld))
            mergeframe=0.5*(curdf.loc[:,"Close"]+curdf.loc[:,"Open"])
            diffd=(curdf.loc[:,"High"]-curdf.loc[:,"Low"]) 
            
            self.sigmadist.extend(((mergeframe-curdf.loc[:,"Low"])/diffd).values.tolist())
            #print ( self.sigmadist[-1])
            #print(mergeframe)
            mergeframe.name=self.symnames[-1]
            diffd.name=self.symnames[-1]

            if (ialld == 0):
                self.values=mergeframe
                self.tottime=mergeframe.shape[0]
                self.range=diffd
            else:
                self.values=pd.concat((self.values,mergeframe),axis=1) 
                self.range=pd.concat((self.range,diffd),axis=1) 

                
            self.corrs[self.symnames[-1]]=pd.Series((pd.read_json(dirname+"corr_"+alld,typ="series")))
        print("Done!")


In [8]:
sd1=stockdata(totsym=50)
stda=((np.array(sd1.sigmadist)))
stda = stda[~np.isnan(stda)]
stdev=stda.std()

Loading...
0 / 50
Done!


In [ ]:
sd=stockdata()


Loading...
0 / 5664
500 / 5664
1000 / 5664
1500 / 5664
2000 / 5664
2500 / 5664
3000 / 5664
3500 / 5664
4000 / 5664
4500 / 5664
5000 / 5664
5500 / 5664


In [3]:
class stockdatagen:
    def __init__(self, data, nchan=10, batch_size=128, lookback=30, topred=5,stdev=0.0,seed=374561):
        self.batch_size=batch_size
        self.nchan=nchan
        self.lookback=lookback
        self.topred=topred
        self.data=data
        self.seed=seed
        self.tottime=data.tottime
        self.values=data.values
        self.range=data.range
        self.corrs=data.corrs
        self.symnames=data.symnames
        if (self.seed==None):
                self.rng = np.random
        else:
                self.rng = np.random.RandomState(self.seed)

    def sreturn(self,start=None,Syms=None):
        xvalbatch=[]
        yvalbatch=[]
        ychanbatch=[]

        maxinc=self.tottime-2*(self.lookback+self.topred)
        
        if Syms==None:
            numpoints=self.batch_size
        else:
            numpoints=len(Syms)
        for bb in range(numpoints):
            if Syms==None:
                cursym=self.rng.choice(np.array(self.symnames))
            else:
                cursym=Syms[bb]
            if start==None:
                starttime=int(random.randint(0, maxinc))
            else:
                starttime=int(start)
            curcorr=self.corrs[cursym].loc[::-1]
            xvalrange=np.abs(self.range.loc[:,cursym].iloc[starttime:starttime+self.lookback])
            xvalsigma=self.rng.normal(loc=0.0, scale=xvalrange*stdev)

            xval=[(self.values.loc[:,cursym].iloc[starttime:starttime+self.lookback]).values+xvalsigma]
            
            yval=[(self.values.loc[:,cursym].iloc[starttime+self.lookback:starttime+self.lookback+self.topred]).values]  
            ychan=[1.0]
            for ic,cc in enumerate( curcorr.index):
                if (ic>=(self.nchan-1)):
                    break
                xval.append ((self.values.loc[:,cc].iloc[starttime:starttime+self.lookback]).values)
                yval.append ((self.values.loc[:,cc].iloc[starttime+self.lookback:starttime+self.lookback+self.topred]).values)
                ychan.append(curcorr[cc])
            xval=np.array(xval)
            yval=np.array(yval)
            norm=xval[:,-1]
            norm=norm.reshape(norm.shape[0],1)
            xval= ((xval)/norm)-1.0
            yval= ((yval)/norm)-1.0

            xvalbatch.append(np.array(xval))
            yvalbatch.append(np.array(yval))
            ychanbatch.append(np.array(ychan))
            
            #gain=ychanbatch.max(axis=1)-xvalbatch[:,-1]
        return(np.array (xvalbatch, dtype="float32"),np.array (yvalbatch, dtype="float32"),np.array (ychanbatch, dtype="float32"))


In [4]:
sdg=stockdatagen(sd,nchan=1, batch_size=128, lookback=30, topred=10,stdev=stdev, seed=374361)
sdg.sreturn()

NameError: name 'sd' is not defined

In [53]:

class LSTM(nn.Module):
    def __init__(self,nchan=4,ntime=100,n_hidden=256,n_layers=64,npoint=5):
        super(LSTM, self).__init__()
        self.n_hidden=n_hidden
        self.n_layers=n_layers
        self.ntime=ntime+npoint
        self.nchan=nchan
        self.npoint=npoint 
        self.output_size=self.nchan*self.npoint
        self.ks=10





        self.lin = nn.Sequential(  
        nn.Linear(464, 2*400)
        )

        self.out = nn.Linear(self.n_hidden*self.npoint, self.output_size)
        
        self.out10 = nn.Linear(self.n_hidden,self.n_hidden)
        self.out11 = nn.Linear(self.n_hidden,self.nchan*self.npoint)


        self.lstm = nn.LSTM(input_size=nchan,hidden_size= self.n_hidden, num_layers=n_layers,batch_first=True)
        self.lstm1 = nn.LSTM(input_size=1,hidden_size= self.n_hidden, num_layers=n_layers,batch_first=True)
        self.lstminter = nn.LSTM(input_size=self.n_hidden,hidden_size= self.n_hidden, num_layers=n_layers,batch_first=True)

        #self.soft = nn.Sequential( nn.Softmax(dim=1))


    def forward(self, inputs):
        inputlstm=inputs[0]
        inputlstmshape=inputlstm.shape
        batchs =inputlstmshape[0]
        inputconv=inputs[1].unsqueeze(1).transpose(1, 2)


        h0c = torch.rand( self.n_layers, batchs, self.n_hidden).to('cuda')  
        c0c = torch.rand( self.n_layers, batchs, self.n_hidden).to('cuda')
        LSc,hiddenc=self.lstm1(inputconv, (h0c,c0c)) 
        print(inputconv.shape)
        print(h0c.shape)
        print(c0c.shape)

        inputlstm=inputlstm.transpose(1, 2)

        #print(LSc.shape)
        #print(inputlstm.shape)
        h0 = torch.rand ( self.n_layers, batchs, self.n_hidden).to('cuda')  
        c0 = torch.rand( self.n_layers, batchs, self.n_hidden).to('cuda')
        LS,hidden=self.lstm(inputlstm, hiddenc) 
        LS=F.relu(LS)
        LS,hidden=self.lstminter(LS, hidden) 
        #output = self.out(torch.cat((LS[:,-1,:],LSc[:,-1,:]),dim=1))
        LS=LS[:,-1,:].reshape(LS.size(0), -1)
        
        
        #output = self.out(LS)
        #return output.reshape(output.shape[0],self.nchan,self.npoint)
        LS = F.relu(self.out10(LS))
        LS = F.relu(self.out10(LS))
        output = self.out11(LS)
        return output.reshape(output.shape[0],self.nchan,self.npoint)
    
    


In [54]:

class Conv(nn.Module):
    def __init__(self,nchan=4,ntime=100,n_hidden=64,n_layers=10,npoint=5):
        super(Conv, self).__init__()
        self.n_hidden=n_hidden
        self.n_layers=n_layers
        self.ntime=ntime+npoint
        self.nchan=nchan
        self.npoint=npoint 
        self.output_size=self.nchan*self.npoint
        self.ks=10

        self.conv1 = nn.Sequential( 
            nn.Conv1d(self.nchan, self.n_hidden,kernel_size=5, padding=1),
            nn.MaxPool1d(2)
        )
        self.conv2 = nn.Sequential( 
            nn.Conv1d(self.n_hidden, self.n_hidden,stride=1, kernel_size=5, padding=1),
            nn.MaxPool1d(2)
        )

        self.lin = nn.Sequential(  
        nn.Linear(self.n_hidden*30, self.n_hidden*2)
        )

        self.out = nn.Linear(2*self.n_hidden, self.nchan)



    def forward(self, inputs):
        batchs =inputs.shape[0]
        CO = F.relu(self.conv1(inputs))  
        CO = F.relu(self.conv2(CO))  
        flat=CO.reshape(CO.size(0), -1)
        #6,14,30 print(flat.shape)

        lin = F.relu(self.lin(flat))

        output = self.out(lin)
        return output.reshape(inputs.shape[0],self.nchan),lin


In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
trsamp={}
nchan=20
batch_size=16
lookback=64
topred=5

trsamp["train"]=stockdatagen(sd,nchan=nchan, batch_size=batch_size, lookback=lookback, topred=topred,stdev=stdev, seed=375363)
trsamp["val"]=trsamp["train"]

plt.close('all')
n_steps=5000
nchantot=nchan
ntime=lookback+topred
nepoch=int(n_steps/5)
biasvsal=0.1
topred=topred









modellstm = LSTM(nchan=nchantot,npoint=topred,ntime=ntime,n_hidden=64,n_layers=2).to('cuda')
modellstm.train()
modelconv = Conv(nchan=nchantot,npoint=topred,ntime=ntime).to('cuda')
modelconv.train()

        
criterion = nn.MSELoss()

optimizerlstm = torch.optim.Adam(modellstm.parameters(), lr=0.005, weight_decay=0.0)
optimizerconv = torch.optim.Adam(modelconv.parameters(), lr=0.0001, weight_decay=0.0)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizerlstm, factor=0.5, patience=0, cooldown=6, verbose=True)

#valsamp=torch.from_numpy(fullval[0]).to('cuda')
#valsamptrue=torch.from_numpy(fullval[1]).to('cuda')
fig, axs = plt.subplots(4)
fig1, axs1 = plt.subplots(2)
plotuple={"loss":[]}
with tqdm(total=n_steps, desc='Start step {}; Training for {} steps'.format(0, n_steps)) as pbar:
        for istep in range(n_steps):
            
                optimizerlstm.zero_grad()
                optimizerconv.zero_grad()
                curtuple = trsamp["train"].sreturn()
                curdata = torch.from_numpy(curtuple[0]).to('cuda')
                #print (curdata.shape)

                outputsconv  = modelconv(curdata)
                predchantrue  = outputsconv[0]
                topass  = outputsconv[1]
                
                outputs  = modellstm((curdata,topass))
                curtrue = torch.from_numpy(curtuple[1]).to('cuda')
                #print(curtrue.shape)

                #maxcurtrue = torch.sum(curtrue, 2)[:,0]/topred
                #print(maxcurtrue.shape)        
                #print(curtrue[:,0,0].shape)    
                #togain=((maxcurtrue-curtrue[:,0,0])/curtrue[:,0,0])
                #print("maxcurtrue")
                #print(maxcurtrue.shape)

                #print(curdata)

                curchantrue = torch.from_numpy(curtuple[2]).to('cuda').float()
                curpred = outputs

                #loss = criterion(curpred,curtrue).mean()

                #print((curpred-togain)/outputs[:,-1])

                
                #print(togain)


                lossconv = criterion(predchantrue,curchantrue).mean()



                curpred.reshape(curtrue.shape)
                #print("curpred")
                #print (curpred[0,0,:])
                #print("curtrue")
                #print (curtrue[0,0,:])
                loss = 100.*criterion(curpred,curtrue)+lossconv
                loss.backward(retain_graph=True)
                #torch.nn.utils.clip_grad_norm_(modellstm.parameters(), 0.1)
                optimizerlstm.step()
                

                #lossconv.backward(retain_graph=False)
                #torch.nn.utils.clip_grad_norm_(modelconv.parameters(), 0.05)
                
                optimizerconv.step()   
                
                pbar.set_postfix(L='{:.4f}'.format(loss),Lchan='{:.4f}'.format(lossconv))
                pbar.update()
                if (istep%nepoch==0 and istep>0):
                        modellstm.eval()
                        modelconv.eval()
                        lateststart= sd.tottime - ntime 
                        fullval=trsamp["val"].sreturn(start=lateststart,Syms=sd.symnames)
                        
                        plotuple["loss"].append(loss.detach().cpu().numpy())
                        valsamp=torch.from_numpy(fullval[0]).to('cuda')
                        valsamptrue=torch.from_numpy(fullval[1]).to('cuda')

                        for param_group in optimizerlstm.param_groups:
                                print(param_group["lr"])
                        for param_group in optimizerconv.param_groups:
                                print(param_group["lr"])

                        
                        outputsconv  = modelconv(valsamp)
                        predchantrue  = outputsconv[0]
                        topass  = outputsconv[1]
                        outputs  = modellstm((valsamp,topass))

                        
                        curtrue = torch.from_numpy(fullval[1]).to('cuda')
                        #maxcurtrue = torch.mean(curtrue, 2)[:,0]
                        #togain=((maxcurtrue-curtrue[:,:,0])/curtrue[:,:,0])[:,0]  
                        curpred = outputs
                        #argm=(np.argsort(togain.detach().cpu().numpy()))

                        #outputs= outputs.reshape(valsamptrue.shape[0],valsamptrue.shape[1])

                        #print (outputs)
                        #print (valsamptrue) 
                        #print (np.array(xvals).shape)
                        #print (np.array(yvals).shape)
                        plot=True
                        if plot:

                                #plt.clf()
        
                                axs[0].cla()
                                axs[1].cla()
                                axs[2].cla()
                                axs[3].cla()
                                #plt.plot(np.append(valsamp.cpu().numpy()[0,0,:],outputs.cpu().detach().numpy()[0,0,:]))
                                #plt.plot(np.append(valsamp.cpu().numpy()[0,1,:],outputs.cpu().detach().numpy()[0,1,:]))
                                bnum=11
                                axs[0].plot(np.append(valsamp.cpu().numpy()[bnum,0,:],outputs.cpu().detach().numpy()[bnum,0,:]))
                                axs[1].plot(np.append(valsamp.cpu().numpy()[bnum,1,:],outputs.cpu().detach().numpy()[bnum,1,:]))
                                axs[2].plot(np.append(valsamp.cpu().numpy()[bnum,2,:],outputs.cpu().detach().numpy()[bnum,2,:]))
                                axs[3].plot(np.append(valsamp.cpu().numpy()[bnum,3,:],outputs.cpu().detach().numpy()[bnum,3,:]))
                                axs[0].plot(np.append(valsamp.cpu().numpy()[bnum,0,:],curtrue.cpu().detach().numpy()[bnum,0,:]))
                                axs[1].plot(np.append(valsamp.cpu().numpy()[bnum,1,:],curtrue.cpu().detach().numpy()[bnum,1,:]))
                                axs[2].plot(np.append(valsamp.cpu().numpy()[bnum,2,:],curtrue.cpu().detach().numpy()[bnum,2,:]))
                                axs[3].plot(np.append(valsamp.cpu().numpy()[bnum,3,:],curtrue.cpu().detach().numpy()[bnum,3,:]))
                                #for ipl in range(10):
                                    #axs[0].plot(np.append(valsamp.cpu().numpy()[argm[-ipl],0,:],valsamptrue.cpu().detach().numpy()[argm[-ipl],0,:]))
                                    #axs[1].plot(np.append(valsamp.cpu().numpy()[argm[ipl],0,:],valsamptrue.cpu().detach().numpy()[argm[ipl],0,:]))
                                #    axs[0].plot((valsamp.cpu().numpy()[argm[-ipl],0,:]))
                                 #   axs[1].plot((valsamp.cpu().numpy()[argm[ipl],0,:]))
                                  #  axs[2].plot((valsamptrue.cpu().detach().numpy()[argm[-ipl],0,:]/valsamptrue.cpu().detach().numpy()[argm[-ipl],0,:].mean()))
                                   # axs[3].plot((valsamptrue.cpu().detach().numpy()[argm[ipl],0,:]/valsamptrue.cpu().detach().numpy()[argm[ipl],0,:].mean()))
                                axs1[0].cla()
                                axs1[0].plot(plotuple["loss"])

                                #plt.xlabel("steps")
                                #plt.ylabel("position")
                                #plt.title("randdwalk")
                                plt.show(block=False)
                                plt.pause(1)
                        modellstm.train()
                        modelconv.train()
                        #sys.exit()
                        #validation_loss=criterion(outputs,valsamptrue).mean()*1000
                        #scheduler.step(validation_loss)
                        #print (validation_loss.tolist())

        plt.close()


NameError: name 'stockdatagen' is not defined